# Pack Padded Sequences

When training RNN (LSTM or GRU or vanilla-RNN), it is difficult to batch the variable length sequences. For example: if the length of sequences in a size 8 batch is [4,6,8,5,4,3,7,8], you will pad all the sequences and that will result in 8 sequences of length 8. You would end up doing 64 computations (8x8), but you needed to do only 45 computations. Moreover, if you wanted to do something fancy like using a bidirectional-RNN, it would be harder to do batch computations just by padding and you might end up doing more computations than required.

Instead, PyTorch allows us to pack the sequence, internally packed sequence is a tuple of two lists. One contains the elements of sequences. Elements are interleaved by time steps (see example below) and other contains the size of each sequence the batch size at each step. This is helpful in recovering the actual sequences as well as telling RNN what is the batch size at each time step. This can be passed to RNN and it will internally optimize the computations.

*[[Source]](https://stackoverflow.com/a/51030945)*

Let's see how it can be implemented using Pytorch step-by-step.

## Convert Text to indexes

Construct word-to-index and index-to-word dictionaries, tokenize words and convert words to indexes. Remember to add special tokens like `<SOS>`, `<EOS>`, `<PAD>`, `<UNK>` and add those tokens and respective indexes to both the above dictionaries as well.

In [1]:
specials = {'<PAD>': 0, '<SOS>':1, '<EOS>': 2, '<UNK>': 3}
data = ['pack', 'padded', 'sequences']

vocab = list(specials.keys()) + sorted(set([token for seq in data for token in seq]))
vectorized_data = [[vocab.index(token) for token in seq] for seq in data]

print(vocab)
print(vectorized_data)

['<PAD>', '<SOS>', '<EOS>', '<UNK>', 'a', 'c', 'd', 'e', 'k', 'n', 'p', 'q', 's', 'u']
[[10, 4, 5, 8], [10, 4, 6, 6, 7, 6], [12, 7, 11, 13, 7, 9, 5, 7, 12]]


## Convert Sequences to Some Fixed Length

Two work with neural networks we need to pad each sequence with some value (`<PAD>` generally). This makes every sequence of same length. The input to the model has shape of (B, N) where B is the batch size and N is the length of each sequence. Python provides `pad_sequence` method to make this job easy for us.

In [2]:
from torch import LongTensor
from torch.nn.utils.rnn import pad_sequence

def pad_collate(batch):
  seq_len = [len(seq) for seq in batch]
  batch = [LongTensor(seq) for seq in batch]
  seq = pad_sequence(batch, batch_first=True, padding_value=specials['<PAD>'])
  return seq, seq_len

In [3]:
padded_seq, seq_len = pad_collate(vectorized_data)
print(seq_len)
print(padded_seq)

[4, 6, 9]
tensor([[10,  4,  5,  8,  0,  0,  0,  0,  0],
        [10,  4,  6,  6,  7,  6,  0,  0,  0],
        [12,  7, 11, 13,  7,  9,  5,  7, 12]])


## Convert Padded Sequences to Embeddings

`padded_seq` is a (B, N) matrix. After introducing embedding of dimension E it subsequently becomes (B, E, N) matrix.

In [4]:
import torch.nn as nn

embedding_dim = 4
embedding = nn.Embedding(len(vocab), embedding_dim)
embedded_seq = embedding(padded_seq)
embedded_seq

tensor([[[-6.8429e-01,  5.8787e-06, -1.3540e+00, -8.6254e-01],
         [ 1.3710e+00,  6.3946e-01, -1.5811e+00,  3.2993e+00],
         [-2.5831e-03, -1.5077e+00, -1.2014e+00, -2.5649e-01],
         [-1.0505e+00, -1.4781e+00, -9.2058e-01,  6.3202e-01],
         [ 1.0947e-02, -1.1987e+00,  1.5961e+00, -2.7746e-01],
         [ 1.0947e-02, -1.1987e+00,  1.5961e+00, -2.7746e-01],
         [ 1.0947e-02, -1.1987e+00,  1.5961e+00, -2.7746e-01],
         [ 1.0947e-02, -1.1987e+00,  1.5961e+00, -2.7746e-01],
         [ 1.0947e-02, -1.1987e+00,  1.5961e+00, -2.7746e-01]],

        [[-6.8429e-01,  5.8787e-06, -1.3540e+00, -8.6254e-01],
         [ 1.3710e+00,  6.3946e-01, -1.5811e+00,  3.2993e+00],
         [-3.4723e-01, -1.0206e+00,  1.7900e+00,  1.0191e+00],
         [-3.4723e-01, -1.0206e+00,  1.7900e+00,  1.0191e+00],
         [ 2.4131e+00, -6.9333e-01, -1.2715e-01,  1.6575e+00],
         [-3.4723e-01, -1.0206e+00,  1.7900e+00,  1.0191e+00],
         [ 1.0947e-02, -1.1987e+00,  1.5961e+00, -2.7

## Pack Padded Sequences

Now its time to pack the padded, embedded sequences. We need to feed the actual length of the sequence (before padding) to inform pytorch when to pack and unpack the sequence. Packing cannot be done before embedding. Pytorch comes with `pack_padded_sequence` method to do the job.

The `packed_seq` and `packed_output` are formats that pytorch rnns can read and ignore the padded inputs when calculating gradients for back-propagation. If we set `enforce_sorted=True`, inputs are sorted by decreasing length.

In [5]:
from torch.nn.utils.rnn import pack_padded_sequence

hid_dim = 8
n_layers = 2

rnn = nn.LSTM(embedding_dim, hid_dim, n_layers, batch_first=True)
packed_seq = pack_padded_sequence(embedded_seq, seq_len, batch_first=True, enforce_sorted=False)
packed_output, hidden = rnn(packed_seq)
print(packed_seq)

PackedSequence(data=tensor([[ 1.8938e-01, -1.1308e-01,  6.7285e-01,  8.1673e-01],
        [-6.8429e-01,  5.8787e-06, -1.3540e+00, -8.6254e-01],
        [-6.8429e-01,  5.8787e-06, -1.3540e+00, -8.6254e-01],
        [ 2.4131e+00, -6.9333e-01, -1.2715e-01,  1.6575e+00],
        [ 1.3710e+00,  6.3946e-01, -1.5811e+00,  3.2993e+00],
        [ 1.3710e+00,  6.3946e-01, -1.5811e+00,  3.2993e+00],
        [-5.0820e-01,  5.1015e-01,  5.4276e-01,  6.1604e-01],
        [-3.4723e-01, -1.0206e+00,  1.7900e+00,  1.0191e+00],
        [-2.5831e-03, -1.5077e+00, -1.2014e+00, -2.5649e-01],
        [-3.6181e-01,  3.3166e-01, -2.2805e-01,  5.7968e-01],
        [-3.4723e-01, -1.0206e+00,  1.7900e+00,  1.0191e+00],
        [-1.0505e+00, -1.4781e+00, -9.2058e-01,  6.3202e-01],
        [ 2.4131e+00, -6.9333e-01, -1.2715e-01,  1.6575e+00],
        [ 2.4131e+00, -6.9333e-01, -1.2715e-01,  1.6575e+00],
        [-6.3850e-01,  2.9805e-01,  8.1755e-01,  1.0044e+00],
        [-3.4723e-01, -1.0206e+00,  1.7900e+00,  1

## Pad Packed Sequences

This returns the padded output with shape (B, O, H) where O is the length of longest output sequence and H is the hidden dimension. The length of each sequence is given by `output_lengths`.

In [6]:
from torch.nn.utils.rnn import pad_packed_sequence

padded_output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)